# Quantum Chemistry Tutorial 5 - Espace actif

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
2. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
April 2024

## <a id='toc1_5_'></a>[Réduction des ressources](#toc0_)

**La réduction des ressources fait référence à la réduction des exigences de calculs pour exécuter un algorithme.** Cela s’applique à la fois à l’informatique classique et quantique. En informatique conventionnelle, cela se traduit par moins de bits requis pour stocker les informations et moins d’opérations à effectuer sur ces bits. Pour l’informatique quantique à modèle de portes, cela se traduit par une réduction du nombre de qubits et du nombre de portes dans les circuits quantiques.

La molécule de dimère d'eau dans la base `6-311G(d,p)` nécessiterait 94 qubits et intrinsèquement un circuit quantique considéré comme trop profond pour les architectures actuelles (dispositifs NISQ). La réduction du nombre de qubits requis pour coder le problème constitue le premier moyen indirect de réduire ces exigences. Les deux méthodes discutées ci-dessous sont la réduction de **l'espace actif** et la **prise en compte des symétries moléculaires**.



### <a id='toc1_5_1_'></a>[Sélection d'espace actif](#toc0_)

La **sélection d'espace actif** *est un moyen de réduire le nombre de qubits en ignorant certaines orbitales moléculaires dans le calcul post-Hartree-Fock*. A cet effet, on classe  les orbitales moléculaires en orbitales doublement occupées, actives et externes :

* les orbitales doublement occupées (**core orbitals**) sont toujours occupées par deux électrons;
* les orbitales actives (**valence orbitals**) peuvent être occupées par zéro, un ou deux électrons.
* les orbitales externes (**virtual orbitals**) ne sont jamais occupées.

<!-- <center><img src="Graphics/Sketch_active_space.png" width="450"/></center> -->
![Sketch_active_space.png](Graphics/Sketch_active_space.png)

Dans cette approximation, un certain nombre d'*électrons actifs* peuvent peupler les *orbitales actives* à partir desquelles nous pouvons générer un espace de taille finie de déterminants de Slater.

**Note**</br>
Le nombre de *spin-orbitales actives* détermine le *nombre de qubits* requis pour effectuer des simulations quantiques de la structure électronique de la molécule. Appliquer cette méthode est trivial, mais choisir les orbitales moléculaires à geler ne l’est pas.

 Bien qu’il n’existe pas d’algorithme général pour effectuer cette opération, on peut évaluer quelles orbitales geler en imprimant d’abord les occupations des orbitales moléculaires.

In [ ]:
DWat_mol.basis = "6-311+G(d,p)"
DWat_mol.build()
DWat_mol.verbose = 0
n_mos = DWat_mol.nao_nr()
print(f'Le nombre d\'orbitales moléculaires, dans la base {DWat_mol.basis}, est {n_mos}')
mf = scf.RHF(DWat_mol)
mf.kernel()
mf.mo_occ

Une façon d’imaginer cela est de considérer uniquement les orbitales frontières (HOMO, LUMO) et leurs voisines. La corrélation perdue sur l’énergie totale en considérant uniquement un sous-ensemble de l’espace actif complet peut être faible. Par exemple, le gel des orbitales moléculaires bassement occupées est connu sous le nom d'**approximation du noyau gelé** ou **frozen-core approximation** et peut être appliqué car les orbitales du coeur ne se mélangent pas avec les orbitales de valence. Il existe également des algorithmes pour geler les orbitales virtuelles, comme la méthode de troncature des orbitales naturelles gelées (FNO) [[Collect. Tchèque. Chimique. Commun. 2005, 70, 837-850](https://doi.org/10.1135/cccc20050837) et [J. Chimique. Phys. 2008, 128, 164101](https://doi.org/10.1063/1.2902285)]. Même si le choix d'un espace actif approprié peut résoudre un problème, cela peut contribuer à réduire *a priori* les ressources quantiques.

La cellule de code ci-dessous aboutit à la prise en compte des orbitales moléculaires proches du niveau HOMO-LUMO. La fonction `get_active_orbitals_with_homo_lumo` aide les utilisateurs à obtenir les étiquettes de ces orbitales. Dans notre cas, l'espace actif est défini du HOMO-3 au LUMO+3.

In [ ]:
def get_active_orbitals_with_homo_lumo(n_mos, mf_pyscf, homo_minus_n=0, lumo_plus_n=0):
    """Function that returns a list of active orbitals if the user wants to
    consider only a subset from HOMO(-homo_min_n) to LUMO(+lumo_plus_n)
    orbitals. Users should be aware of degeneracies, as this function does not
    take this property into account.

    Args:
        n_mos: Number of molecular orbitals
        mf_pyscf (mf object): pyscf meam-field object of the molecule
        homo_minus_n (int): Starting point at HOMO - homo_minus_n
        lumo_plus_n (int): Ending point at LUMO + lumo_plus_n

    Returns:
        list of int: orbitals in the active space.
    """

    # Index of HOMO and LUMO
    lumo_idx = mf_pyscf.mo_occ.tolist().index(0.) # mf_pyscf.nelectron//2
    homo_idx = lumo_idx - 1 # mf_pyscf.nelectron//2 - 1

    active_orbitals = homo_idx-homo_minus_n, lumo_idx+lumo_plus_n+1

    return active_orbitals


In [ ]:
# Selecting HOMO-3 to LUMO+3 orbitals
active_space = get_active_orbitals_with_homo_lumo(mol_psyf.nao, mf, homo_minus_n=3, lumo_plus_n=3)
n_active_mos = active_space[1]-active_space[0]
print(f'Le nombre de MOs dans l\'espace actif est {n_active_mos} et les indexes correspondants sont {list(range(active_space[0],active_space[1]))}')

In [ ]:
frozen_orbitals = [n for n in range(n_mos) if n not in range(active_space[0],active_space[1])]

En suivant cette approche, nous réduisons le nombre d'orbitales considérées de 68 à 8. Avec les mappages fermions-qubits conventionnels, le nombre de qubits peut être réduit de 94 à 16. Bien qu'il s'agisse d'une réduction impressionnante, les configurations impliquant des orbitales inférieures à HOMO- 4 et supérieurs, LUMO+4 ne sont pas pris en compte ici. Cela se traduit par une énergie plus éloignée de la véritable énergie car rien ne nous dit que ces configurations ne sont pas pertinentes *a priori*. Dans la cellule de code suivante, nous considérons le calcul CCSD du dimère d'eau avec et sans ces orbitales.

In [ ]:
from pyscf import cc

#
# Without Frozen orbitals
#
mycc = cc.CCSD(mf)
mycc.kernel()
print('CCSD correlation energy', mycc.e_tot)

#
# With Frrozen orbitals 
#
mycc_fr = cc.CCSD(mf)
mycc_fr.frozen = frozen_orbitals
mycc_fr.kernel()
print('CCSD correlation energy', mycc_fr.e_tot)

print(f"Energy difference: {abs(mycc.e_tot-mycc_fr.e_tot)*627.5} kcal/mol.")

Ce choix se traduit par une différence d'énergie de plus de 310 kcal/mol, bien supérieure à [précision chimique](https://en.wikipedia.org/wiki/Computational_chemistry) (1 kcal/mol). Dans cette situation, la limite inférieure avec l’espace actif réduit est supérieure à l’état fondamental du CCSD. Ce qu’il faut retenir, c’est d’être prudent lors du choix d’un espace actif pour une molécule.

Restreignons maintenant l'espace actif à l'HOMO et au LUMO.

In [ ]:
#
# Without Frozen orbitals
#
mycc = cc.CCSD(mf)
mycc.kernel()
print('CCSD correlation energy', mycc.e_tot)

#
# With Frrozen orbitals 
#
mycc_fr = cc.CCSD(mf)
mycc_fr.frozen = (0,homo_idx)
mycc_fr.kernel()
print('CCSD correlation energy', mycc_fr.e_tot)

print(f"Energy difference: {abs(mycc.e_tot-mycc_fr.e_tot)*627.5} kcal/mol.")

On constate une amélioration du résultat avec une différence d'énergie de plus de 71.73 kcal/mol.